# **Lab 1: Matrix Factorization**
**Lovisa Strange**

# **Abstract**

In this lab report, three algorithms related to matrix factorization are presented.

#**About the code**

A short statement on who is the author of the file, and if the code is distributed under a certain license.

In [ ]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2024 Lovisa Strange (lastrange@kth.se)

# This file is part of the course DD2365 Advanced Computation in Fluid Mechanics
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them.

In [1]:
# Load neccessary modules.
from google.colab import files

import time
import numpy as np

#try:
#    from dolfin import *; from mshr import *
#except ImportError as e:
#    !apt-get install -y -qq software-properties-common
#    !add-apt-repository -y ppa:fenics-packages/fenics
#    !apt-get update -qq
#    !apt install -y --no-install-recommends fenics
#    from dolfin import *; from mshr import *

#import dolfin.common.plotting as fenicsplot

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D

# **Introduction**

Give a short description of the problem investigated in the report, and provide some background information so that the reader can understand the context. If appropriate, include a discussion on the state of the art.

Briefly describe what method you have chosen to solve the problem, and justify why you selected that method.

Here you can express mathematics through Latex syntax, and use hyperlinks for references.

[Hyperlink to DD2363 course website.](https://kth.instructure.com/courses/17068)

$
{\displaystyle \frac{\partial u}{\partial t}} + u\cdot \nabla u +\nabla p = f, \quad \nabla \cdot u=0$



# **Method**

Describe the methods you used to solve the problem. This may be a combination of text, mathematical formulas (Latex), algorithms (code), data and output.  

##Sparse matrix-vector product
For matrices that are sparse, ie largely consist of zeros, it is often not efficient to use regular matrix-vector multiplication. Instead, we can use another algoritm that uses a specific data structure to make the computation more efficient.

This data structure is called *compressed row structure* (lecture X, slide Y) or CRS. A matrix is represented as 3 arrays. The first one conatins all non-zero elements of the matrix: The seccond one (of the same length) has the column index for each of the non-zero elements. Lastly, the third array contains the index that points to the position in the first array where each new row starts.

We look at an example using the matrix
$$A= \begin{bmatrix}3&0&0\\0&2&0 \end{bmatrix}.$$
This matrix would be reprecented with the values $$
val=[3,2],$$
the column indicies $$
col\_idx = [0,1]
$$
and the row pointers $$
row\_ptr=[0,1,2].
$$

We can find the CRS representation of a given matrix by looping over its rows and saving the non-zero elements and their columns, as well as keeping track of when a new row is started. This is done in the code below.

In [19]:
# CRS representation of A

class CRS:
  def __init__(self,A):
    val = []
    col_idx = []
    row_ptr = []

    for row in A:
      colCounter = 0
      row_ptr.append(len(val))

      for elem in row:
        if elem != 0:
          val.append(elem)
          col_idx.append(colCounter)
        colCounter +=1
    row_ptr.append(len(val))


    self.val = val
    self.col_idx = col_idx
    self.row_ptr = row_ptr

Then, we can use this data structure to compute the product between a sparse matrix and a vector more efficiently. (Lecture X, slide Y) To understand how this works, we will lok at an example.

Using the regular matrix-vector product, we would do the following computations
$$c =A\cdot b = \begin{bmatrix}3&0&0\\0&2&0 \end{bmatrix} \begin{bmatrix}1\\2\\3 \end{bmatrix}= \begin{bmatrix}3⋅1+0\cdot2+0⋅3\\0⋅1+2⋅2+0⋅3 \end{bmatrix} = \begin{bmatrix}3\\4 \end{bmatrix}$$

If we instead store the matrix using CRS, we loop over each row, and then look at the non-zero elements of that row and multiply them with the corresponding element in the vector. We get for the same A as above that for the first row we compute
$$
c[0] = val[0]⋅b[0]= 3⋅1 =3
$$

and for the seccond row we get

$$
c[1] = val[1]⋅b[1] =2⋅2=4.
$$

This means that we don't have to compute the multiplication with the zeroes, which is useful for sparse matrices.

The algorithm for this is presented below.

In [40]:
# Sparse matrix vector multiplucation

## Input: vector x with length n, CRS representation of mxn matrix A
## Output: b = Ax

def sparse_matrix_vector_product(A,x):

  b= np.empty(len(x)) # result
  for i in range(len(x)):
    b[i] = 0
    for j in range(A.row_ptr[i],A.row_ptr[i+1]):
      b[i] += A.val[j]*x[A.col_idx[j]]

  return b

##QR factorization

One way in which we can factorize matrices is

In [7]:
# QR - factrorization

## Input: Matrix A
## Output: Q orthogonal, R upper triangular so that A = QR

def QR_factorization(A):
  n = A.shape[0]
  R= np.zeros([n,n])
  Q= np.zeros([n,n])
  v = np.zeros(n)

  for j in range(n):
    v[:] = A[:, j]
    for i in range(j):
      R[i,j] = np.dot(Q[:,i],v[:])
      v[:] = v[:] - R[i,j]*Q[:,i]
    R[j,j] = np.sqrt(sum([elem**2 for elem in v]))
    Q[:,j] = v[:]/R[j,j]

  return Q,R

##Solving Ax=b

In [2]:
# Backward substitution

## Input: Upper triangular matrix U, vector b
## Output: solution to Ux = b

def backward_substitution(U,b):
  n = len(b)
  x = np.zeros(n)
  x[n-1] = b[n-1]/U[n-1,n-1]
  for i in reversed(range(n-1)):
    print(i)
    sum = 0
    for j in range(i+1, n):
      sum += U[i,j]*x[j]
    x[i] = (b[i]-sum)/U[i,i]

  return x

In [5]:
# Solving Ax = b

## Input: matrix A, vector b
## Output: Solution b

def solver(A,b):
  Q,R = QR_factorization(A) # x = (QR)^(-1)b = Q^(-1)R^(-1)b

  partialSol = backward_substitution(R,b)
  QInverse = np.transpose(Q)

  x = np.matmul(QInverse,partialSol)

  return x



# **Results**

##Sparse matrix-vector product


In [47]:
x = np.array([1,1,1,1,1,1])
A = np.array([[3,2,0,2,0,0],[0,2,1,0,0,0],[0,0,1,0,0,0],[0,0,3,2,0,0],[0,0,0,0,1,0],[0,0,0,0,2,3]])
A_sparse = CRS(A)

b_sparse = sparse_matrix_vector_product(A_sparse,x)
print(np.array(b_sparse))

b_dense = np.matmul(A,x)
print(b_dense)

[7. 3. 1. 5. 1. 5.]
[7 3 1 5 1 5]


##QR factorization


In [16]:
A = np.array([[2, -1],[-1,2]])
Q, R = QR_factorization(A)

print("Q = ", Q)
print("R = ", R)
print("A = QR = ", np.matmul(Q,R))
print()
QTQ =  np.matmul(np.transpose(Q),Q)
I = np.identity(A.shape[0])
print("||Q^TQ - I||_F = ", np.linalg.norm(QTQ - I,'fro'))

QR = np.matmul(Q,R)
print("||QR - A||_F = ", np.linalg.norm(QR - A,'fro'))

Q =  [[ 0.89442719  0.4472136 ]
 [-0.4472136   0.89442719]]
R =  [[ 2.23606798 -1.78885438]
 [ 0.          1.34164079]]
A = QR =  [[ 2. -1.]
 [-1.  2.]]

||Q^TQ - I||_F =  2.6901577681355055e-16
||QR - A||_F =  0.0


##Solving Ax = b


In [15]:
A = np.array([[1, 3],[0,1]])
b = np.array([1,1])

x = solver(A,b)
y = np.array([-2,1])
print(x)
print(np.linalg.norm(np.matmul(A,x)-b))
print(np.linalg.norm(x-y))




0
[-2.  1.]
0.0
0.0


Present the results. If the result is an algorithm that you have described under the *Methods* section, you can present the data from verification and performance tests in this section. If the result is the output from a computational experiment this is where you present a selection of that data.

# **Discussion**

Summarize your results and your conclusions. Were the results expected or surprising. Do your results have implications outside the particular problem investigated in this report?